In [2]:
#importing packages needed
import requests
import pandas as pd
import json
import creds

### Accessing User Data File 

Because this is a project showing the possibilities, our user data is generated from an API instead of a file export from a CRM. 

In [3]:
#accessing the API to get user data
r = requests.get('https://peoplegeneratorapi.live/api/person/10')
print(r.status_code)
print(r.json())

200
[{'name': 'Tommie Murray', 'age': 14, 'job': 'Student', 'incomeUSD': 0, 'creditScore': 0, 'ccNumber': None, 'married': False, 'hasChildren': False, 'height': 151.0, 'weight': 70.7, 'eyeColor': 'GRAY', 'email': 'tommie_murray@hotmail.com', 'gender': 'Male', 'hasDegree': False, 'bloodType': 'B+', 'username': 'tommie98', 'politicalLeaning': 0.11, 'religion': 'Islam', 'address': {'streetAddress': '62414 Yundt Forest', 'city': 'Råsunda', 'state': 'Västra Götaland', 'country': 'Sweden', 'zip': '1104', 'geonameId': 2692475, 'phoneNumber': '+46 722758545', 'ipAddress': '66.175.83.3', 'countryCode': 'SE'}, 'doB': 'Tue Sep 22 10:02:51 UTC 2009', 'gpa': 1.7}, {'name': 'Darrel King', 'age': 43, 'job': 'Production Sound Mixer', 'incomeUSD': 79846, 'creditScore': 797, 'ccNumber': '4592-0533-7129-5176', 'married': True, 'hasChildren': False, 'height': 160.0, 'weight': 59.0, 'eyeColor': 'BROWN', 'email': 'king@yahoo.com', 'gender': 'Male', 'hasDegree': False, 'bloodType': 'A-', 'username': 'darrel

In [4]:
#viewing the data as a dataframe
data=r.json()
df = (pd.DataFrame(data))
df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,email,gender,hasDegree,bloodType,username,politicalLeaning,religion,address,doB,gpa
0,Tommie Murray,14,Student,0,0,None,False,False,151.0,70.7,...,tommie_murray@hotmail.com,Male,False,B+,tommie98,0.11,Islam,"{'streetAddress': '62414 Yundt Forest', 'city'...",Tue Sep 22 10:02:51 UTC 2009,1.7
1,Darrel King,43,Production Sound Mixer,79846,797,4592-0533-7129-5176,True,False,160.0,59.0,...,king@yahoo.com,Male,False,A-,darrel61,-0.88,Christianity,"{'streetAddress': '538 Fisher Camp', 'city': '...",Sat Jul 04 03:54:53 UTC 1981,2.6
2,Alexis Bashirian,8,Student,0,0,None,False,False,155.0,49.5,...,alexisbashirian@yahoo.com,Female,True,AB+,alexis82,0.65,Christianity,"{'streetAddress': '2866 Renaldo Ford', 'city':...",Sun May 22 00:23:53 UTC 2016,2.6
3,Stephen Deckow,93,Retired,0,548,6011-4709-0334-0387,False,False,160.0,60.5,...,stephen@gmail.com,Female,True,A+,stephen70,-0.22,Christian,"{'streetAddress': '617 Stanton Hills', 'city':...",Sat Dec 20 16:59:41 UTC 1930,3.1
4,Denis Kutch,4,Student,0,0,None,False,False,166.0,69.3,...,deniskutch@hotmail.com,Male,False,A+,denis73,0.19,Islam,"{'streetAddress': '0087 Zulauf Shoals', 'city'...",Wed Jul 01 22:13:44 UTC 2020,1.6


In [5]:
#getting a list of all the column names

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              10 non-null     object 
 1   age               10 non-null     int64  
 2   job               10 non-null     object 
 3   incomeUSD         10 non-null     int64  
 4   creditScore       10 non-null     int64  
 5   ccNumber          6 non-null      object 
 6   married           10 non-null     bool   
 7   hasChildren       10 non-null     bool   
 8   height            10 non-null     float64
 9   weight            10 non-null     float64
 10  eyeColor          10 non-null     object 
 11  email             10 non-null     object 
 12  gender            10 non-null     object 
 13  hasDegree         10 non-null     bool   
 14  bloodType         10 non-null     object 
 15  username          10 non-null     object 
 16  politicalLeaning  10 non-null     float64
 17  

Since we know we are only going to market this product to adults, we need to remove all children from the dataset 

In [6]:
#removing individuals under the age of 18 
df.drop(df[df['age'] < 18].index, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 1 to 9
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              6 non-null      object 
 1   age               6 non-null      int64  
 2   job               6 non-null      object 
 3   incomeUSD         6 non-null      int64  
 4   creditScore       6 non-null      int64  
 5   ccNumber          6 non-null      object 
 6   married           6 non-null      bool   
 7   hasChildren       6 non-null      bool   
 8   height            6 non-null      float64
 9   weight            6 non-null      float64
 10  eyeColor          6 non-null      object 
 11  email             6 non-null      object 
 12  gender            6 non-null      object 
 13  hasDegree         6 non-null      bool   
 14  bloodType         6 non-null      object 
 15  username          6 non-null      object 
 16  politicalLeaning  6 non-null      float64
 17  r

As we can see the file is now smaller and excludes anyone under the age of 18. 

Since the BMI API we are using requires height in inches and weight in pounds, we will need to update this in our dataframe as well. Currently, the Height is in CM and the Weight is in KG. 

Height: Convert from CM to In

Weight: Convert from KG to LB

To convert from CM to IN, I need to divide the Height column values by 2.54 in order to get the correct value in inches 
To convert from KG to LB, I need to multiply the Weight column values by 2.205 in order to get the correct value in pounds

In [7]:
#adding height in inches and weight in pounds
df['height_inches'] = df['height'] / 2.54

df['weight_pounds'] = df['weight'] * 2.205

In [8]:
#generating just the columns needed to get BMI
bmi_columns_needed = df[['age','height','weight','height_inches','weight_pounds']]
print(bmi_columns_needed)

   age  height  weight  height_inches  weight_pounds
1   43   160.0    59.0      62.992126       130.0950
3   93   160.0    60.5      62.992126       133.4025
5   56   155.0    60.3      61.023622       132.9615
6   23   166.0    44.9      65.354331        99.0045
8   30   166.0    69.3      65.354331       152.8065
9   52   155.0    60.8      61.023622       134.0640


### Appending in BMI Data

In [12]:
def calculate_bmi(row):
    url = "https://fitness-calculator.p.rapidapi.com/bmi"
    querystring = {
        "age": row['age'],
        "weight": row['weight'],
        "height": row['height']
    }
    headers = {
        "X-RapidAPI-Key": creds.api_key,
        "X-RapidAPI-Host": "fitness-calculator.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    # Check if 'data' key is present in the API response
    if 'data' in data:
        bmi = data['data'].get('bmi', None)
        health = data['data'].get('health', None)
        healthy_bmi_range = data['data'].get('healthy_bmi_range', None)
        return bmi, health, healthy_bmi_range
    else:
        return None, None, None  # Return None in case of missing or unexpected data

# Add new columns 'bmi', 'health', and 'healthy_bmi_range' to the DataFrame
df['bmi'], df['health'], df['healthy_bmi_range'] = zip(*df.apply(calculate_bmi, axis=1))

# Display the updated DataFrame
df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,religion,address,doB,gpa,height_inches,weight_pounds,bmi,health,healthy_bmi_range,income level
1,Darrel King,43,Production Sound Mixer,79846,797,4592-0533-7129-5176,True,False,160.0,59.0,...,Christianity,"{'streetAddress': '538 Fisher Camp', 'city': '...",Sat Jul 04 03:54:53 UTC 1981,2.6,62.992126,130.0950,23.05,Normal,18.5 - 25,Middle income
3,Stephen Deckow,93,Retired,0,548,6011-4709-0334-0387,False,False,160.0,60.5,...,Christian,"{'streetAddress': '617 Stanton Hills', 'city':...",Sat Dec 20 16:59:41 UTC 1930,3.1,62.992126,133.4025,NaN,None,None,Low income
5,Leo Frami,56,Speech-Language Pathologist,80550,845,6759-7148-5154-6794-940,False,False,155.0,60.3,...,Islam,"{'streetAddress': '7222 Flatley Lodge', 'city'...",Tue Aug 20 16:32:36 UTC 1968,0.0,61.023622,132.9615,25.10,Overweight,18.5 - 25,Middle income
6,Della Bailey,23,Artificial Intelligence Developer,122420,404,6706834817388198,False,False,166.0,44.9,...,Islam,"{'streetAddress': '72844 Brent Shoals', 'city'...",Fri Aug 17 06:31:27 UTC 2001,3.1,65.354331,99.0045,16.29,Moderate Thinness,18.5 - 25,Middle income
8,Virgina Hyatt,30,Database Administrator,93791,628,6007-2244-9731-6683,False,False,166.0,69.3,...,Hinduism,"{'streetAddress': '0657 Kerry Bypass', 'city':...",Wed Aug 04 11:39:35 UTC 1993,2.8,65.354331,152.8065,25.15,Overweight,18.5 - 25,Middle income


Now that I have added in BMI, Health indicator, and the healthy BMI range, I would like to make my data set more robust to make insights easier. First, I will take the income column and add a new range that signifies lower, middle, or upper class. 

In [10]:
def categorize_income_level(incomeUSD):
    income = int(incomeUSD)  # Convert income to an integer
    
    if income < 52200:
        return 'Low income'
    elif 52200 <= income < 156600:
        return 'Middle income'
    else:
        return 'Upper income'

# Add a new column 'income level' to the DataFrame with the income level categories
df['income level'] = df['incomeUSD'].apply(categorize_income_level)

print(df)


             name  age                                job  incomeUSD  \
1     Darrel King   43             Production Sound Mixer      79846   
3  Stephen Deckow   93                            Retired          0   
5       Leo Frami   56        Speech-Language Pathologist      80550   
6    Della Bailey   23  Artificial Intelligence Developer     122420   
8   Virgina Hyatt   30             Database Administrator      93791   
9   Marlena Kunde   52                         Astronomer      89973   

   creditScore                 ccNumber  married  hasChildren  height  weight  \
1          797      4592-0533-7129-5176     True        False   160.0    59.0   
3          548      6011-4709-0334-0387    False        False   160.0    60.5   
5          845  6759-7148-5154-6794-940    False        False   155.0    60.3   
6          404         6706834817388198    False        False   166.0    44.9   
8          628      6007-2244-9731-6683    False        False   166.0    69.3   
9        